In [2]:
# uncomment pip install in the first run
# !pip install geopy  # streets to longitude/latitude
# !pip install folium  # locations on map
# !pip install bs4  # BeautifulSoup
# !pip install opencage  # geocoder
# !pip install haversine  # distance between two location in km

import requests
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from opencage.geocoder import OpenCageGeocode
import haversine as hs

In [3]:
# credentials and global variables 
CLIENT_ID = ''  # Foursquare ID
CLIENT_SECRET = ''
ACCESS_TOKEN = ''
VERSION = ''
KEY = ''  # GeoCode key
LOCATION = Nominatim(user_agent="foursquare_agent").geocode('Košice')
LAT = LOCATION.latitude
LNG = LOCATION.longitude
GEOCODER = OpenCageGeocode(KEY)

### Streets (borough and location)

In [4]:
# download list from website, keep only relevant columns
streets = pd.concat(pd.read_html('https://www.kosice.sk/mesto/katalog-ulic',
                                 encoding='utf-8'))
streets.drop(columns={'VZN', 'PSČ', 'Poznámka'}, errors='ignore', inplace=True)
streets.rename(columns={'Názov ulice': 'Street', 'Mestská časť': 'Borough'},
               inplace=True)
streets.head()

,Street,Borough
0,Abovská,Košice - Barca
1,Adamova,Košice - Krásna
2,Adlerova,Košice - Dargovských hrdinov
3,Agátová,Košice - Košická Nová Ves
4,Alejová,Košice - Juh


In [ ]:
# pair location to the streets using GeoCage (928 streets, time consuming)
lat = []
lng = []
for i, s in streets.iterrows():
    query = s['Street'] + ', ' + s['Borough'] + ', Slovakia'
    # print(query)
    location = GEOCODER.geocode(query)
    lat.append(location[0]['geometry']['lat'])
    lng.append(location[0]['geometry']['lng'])
streets.insert(2, 'lat', lat, True)
streets.insert(2, 'lng', lng, True)

In [12]:
# remove outliers
streets_outliers = ['Čermeľské údolie', 'Prešovská cesta', 'Ulica Zelený dvor',
                    'Bankov', 'Ulica Sajkov', 'Cesta do Hanisky',
                    'K zoologickej záhrade', 'Bielych albatrosov', 'Bahýľova',
                    'Svetlá pusta', 'Ludvíkov dvor', 'K letisku']

# visually verify street's locations
venues_map = folium.Map(location=[LAT, LNG], zoom_start=12)
for lat, lng, label in zip(streets.lat, streets.lng, streets.Street):
    if label in streets_outliers and hs.haversine((LAT, LNG), (lat, lng)) > 3:
        folium.CircleMarker([lat, lng], radius=5, color='red',
                            popup=label).add_to(venues_map)
    else:
        folium.CircleMarker([lat, lng], radius=5,
                            popup=label).add_to(venues_map)
venues_map

In [26]:
# remove outliers
for i, s in streets.iterrows():
    if s.Street in streets_outliers and hs.haversine((LAT, LNG), (s.lat, s.lng)) > 3:
        streets.drop(i, inplace=True)

# visually verify street's locations and save it
venues_map = folium.Map(location=[LAT, LNG], zoom_start=12)
for lat, lng, label in zip(streets.lat, streets.lng, streets.Street):
    folium.CircleMarker([lat, lng], radius=5, popup=label).add_to(venues_map)
streets.to_csv("streets.csv")
venues_map

### Boroughs (location and population)

In [27]:
# location of boroughs (based on streets)
# streets = pd.read_csv("streets.csv")
boroughs = pd.DataFrame(set(streets['Borough'].tolist()), columns={'Borough'})
lat = []
lng = []
for i, s in boroughs.iterrows():
    location = GEOCODER.geocode(s['Borough'])
    lat.append(location[0]['geometry']['lat'])
    lng.append(location[0]['geometry']['lng'])
boroughs.insert(1, 'lat', lat, True)
boroughs.insert(1, 'lng', lng, True)

# join population
population = pd.concat(pd.read_html(
    io='https://sk.wikipedia.org/wiki/Mestsk%C3%A9_%C4%8Dasti_Ko%C5%A1%C3%ADc',
    match='Počet obyvateľov', encoding='utf-8'))
population.rename(columns={'Poradie': 'Order', 'Mestská časť': 'Borough',
                           'Okres': 'District', 'Plocha v km²': 'Size',
                           'Počet obyvateľov': 'Population'}, inplace=True)
population['Borough'] = 'Košice - ' + population['Borough']
boroughs = boroughs.join(population.set_index('Borough'), on='Borough')
boroughs['Population'] = boroughs['Population'].str.replace(' ', '').astype(int)
boroughs.to_csv('boroughs.csv')
boroughs.head()

,Borough,lng,lat,Order,District,Size,Population
0,Košice - Barca,21.265998,48.676597,12.0,Košice IV,1813,3677
1,Košice - Kavečany,21.206767,48.774899,18.0,Košice I,1005,1337
2,Košice - Myslava,21.203971,48.708800,16.0,Košice II,701,2399
3,Košice - Dargovských hrdinov,21.282077,48.738852,2.0,Košice III,1109,26004
4,Košice - Džungľa,21.270358,48.736222,22.0,Košice I,47,703


In [28]:
# boroughs on map, size reflects population
# boroughs = pd.read_csv('boroughs.csv')
venues_map = folium.Map(location=[LAT, LNG], zoom_start=12)
for lat, lng, label, size in zip(boroughs.lat, boroughs.lng, boroughs.Borough,
                                 boroughs.Population):
    folium.CircleMarker([lat, lng], radius=size/1000, color='green',
                        popup=label+' '+str(size)).add_to(venues_map)
venues_map

### Sporting facilities using Foursquare

In [29]:
# find all sporting facilities in each borough
search_query = ['gym', 'arena', 'stadium', 'tennis', 'hockey', 'squash',
                'bowling', 'golf', 'bedminton']
columns = ['id', 'name', 'location.lat', 'location.lng']
facilities = pd.DataFrame([], columns=columns)
for q in search_query:
    print('Searching ' + q)
    for lat, lng in zip(boroughs.lat, boroughs.lng):
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&\
                client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}'\
                .format(CLIENT_ID, CLIENT_SECRET, lat, lng,
                        ACCESS_TOKEN, VERSION, q, 2000)
        results = requests.get(url).json()
        if results['response']['venues']:
            venues = results['response']['venues']
            facilities = facilities.append(pd.json_normalize(venues)[columns])

facilities.rename(columns={'location.lat': 'lat', 'location.lng': 'lng'},
                  inplace=True)
facilities = facilities.drop_duplicates()

Searching gym
Searching arena
Searching stadium
Searching tennis
Searching hockey
Searching squash
Searching bowling
Searching golf
Searching bedminton


In [32]:
# remove rows with forbidden substrings and save as csv
facilities = facilities[~facilities['name'].str.contains('Gymnázium')]
facilities = facilities[~facilities['name'].str.contains('gymnázium')]
facilities = facilities[~facilities['name'].str.contains('Shop')]

# add some other facilities not found by Foursquare (source=www.sportoviska.sk)
facilities.drop(columns={'Borough'}, errors='ignore', inplace=True)
data = [
        ['1', 'City Gym', 48.729583, 21.288412],
        ['2', 'Vodnolyžiarský vlek Trixen', 48.689345, 21.28447],
        ['3', 'MFK Ťahanovce', 48.757462, 21.255165],
        ['4', 'Baddy Fitness', 48.7528863, 21.2692675],
        ['5', 'Baddy Fitness', 48.7084648, 21.2362396],
        ['6', 'Game bar bowling', 48.6822057, 21.2894243],
       ]
facilities_manually = pd.DataFrame(data, columns=facilities.columns)
facilities = facilities.append(facilities_manually)
facilities.head()

,id,name,lat,lng
0,59e756f2772fbc1b46a4771a,Horvi’s Gym,48.679925,21.296042
1,5fc12266439ae441ad383414,Gymbeam,48.690421,21.259739
0,5112b014e4b01f7081b61fb8,Olymp GYM,48.716127,21.213273
1,50a28f93e4b0225d6363306b,DaWell Gym,48.718742,21.231872
4,515d8372e4b0ff174896c637,Gymnastická hala - Sokol Gymnastik Košice,48.713933,21.226048


In [33]:
# find borough for each facility (as closest street)
streets = pd.read_csv("streets.csv")
bor = []
for i, f in facilities.iterrows():
    b = ""
    min = 100
    for j, s in streets.iterrows():
        dist = hs.haversine((f.lat, f.lng), (s.lat, s.lng))
        if dist < min:
            min = dist
            b = s['Borough']
    bor.append(b)
facilities['Borough'] = bor
facilities.to_csv("facilities.csv")
facilities.head()

,id,name,lat,lng,Borough
0,59e756f2772fbc1b46a4771a,Horvi’s Gym,48.679925,21.296042,Košice - Nad jazerom
1,5fc12266439ae441ad383414,Gymbeam,48.690421,21.259739,Košice - Juh
0,5112b014e4b01f7081b61fb8,Olymp GYM,48.716127,21.213273,Košice - Sídlisko KVP
1,50a28f93e4b0225d6363306b,DaWell Gym,48.718742,21.231872,Košice - Západ
4,515d8372e4b0ff174896c637,Gymnastická hala - Sokol Gymnastik Košice,48.713933,21.226048,Košice - Západ


In [34]:
facilities = pd.read_csv("facilities.csv")
venues_map = folium.Map(location=[LAT, LNG], zoom_start=12)
for lat, lng, name in zip(facilities.lat, facilities.lng, facilities.name):
    folium.CircleMarker([lat, lng], radius=5).add_to(venues_map)
venues_map

### Borough with the highest demand (lowers facilities per population)

In [40]:
# join count of facilities according to its borough
boroughs = boroughs.drop(boroughs[['Facilities']], axis=1, errors='ignore')
stat = facilities.groupby('Borough', as_index=False).count()[['Borough', 'name']]
stat.rename(columns={'name': 'Facilities'}, inplace=True)
boroughs = boroughs.join(stat.set_index('Borough'), on='Borough').fillna(0)
boroughs.to_csv("boroughs.csv")

# find the few ones with highest demand
boroughs['Demand'] = (boroughs['Population']/1000/boroughs['Facilities'])
top_borough = boroughs[boroughs.Facilities > 0]\
                .sort_values(by=['Demand'], ascending=False).head(5)
top_borough.to_csv("top_borough.csv")
top_borough

,Borough,lng,lat,Order,District,Size,Population,Demand,Facilities
3,Košice - Dargovských hrdinov,21.282077,48.738852,2.0,Košice III,1109,26004,13.002000,2.0
10,Košice - Sídlisko Ťahanovce,21.268293,48.755716,6.0,Košice I,826,22129,11.064500,2.0
8,Košice - Nad jazerom,21.288530,48.686518,3.0,Košice IV,366,24613,8.204333,3.0
6,Košice - Sídlisko KVP,21.213507,48.713881,4.0,Košice II,178,23603,7.867667,3.0
17,Košice - Šaca,21.171212,48.631824,10.0,Košice II,4121,5961,5.961000,1.0


### Best street for new sporting facility

In [78]:
# should be in maximal distance from other facilites but within 1km of borough
# it's min-max problem - find distance to the closest facility of each street
# and then select street with maximal value
def find_street(top_borough):
    maxdist = 0
    top_streets = streets.loc[streets['Borough'] == top_borough['Borough']]
    top_facilities = facilities.loc[facilities['Borough'] == top_borough['Borough']]
    top_street = []
    for j, s in top_streets.iterrows():
        # only streets within 1 km distance from borough's center
        bdist = hs.haversine((top_borough.lat, top_borough.lng), (s.lat, s.lng))
        if bdist <= 1:
            mindist = 100
            for i, f in top_facilities.iterrows():
                dist = hs.haversine((f.lat, f.lng), (s.lat, s.lng))
                if dist < mindist:
                    mindist = dist
            if mindist > maxdist:
                maxdist = mindist
                top_street = s['Street']

    top = streets.loc[streets['Street'] == top_street]
    print('Best place for {}: {} street ({}, {}).'
          .format(top_borough['Borough'], top.iloc[0]['Street'], 
                  top.iloc[0]['lat'], top.iloc[0]['lng']))

    # finally visualize the current facilities in top borough (blue color)
    # and the proposal for the best place for the new facility with a red color
    venues_map = folium.Map(location=[top_borough['lat'],
                                      top_borough['lng']], zoom_start=14)
    folium.CircleMarker([top.iloc[0]['lat'], top.iloc[0]['lng']], radius=5,
                        color='red', popup='TOP place').add_to(venues_map)
    for lat, lng, name in zip(top_facilities.lat, top_facilities.lng,
                              top_facilities.name):
        folium.CircleMarker([lat, lng], radius=5, color='blue', 
                            popup=name).add_to(venues_map)
    return venues_map

In [79]:
# top 3
top3 = boroughs[boroughs.Facilities > 0]\
                .sort_values(by=['Demand'], ascending=False).head(3)
find_street(top3.iloc[0])

Best place for Košice - Dargovských hrdinov: Adlerova street (48.744975, 21.2789492).


In [80]:
find_street(top3.iloc[1])

Best place for Košice - Sídlisko Ťahanovce: Austrálska trieda street (48.761846000000006, 21.2608134).
